In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import os
import sys
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def compute_prior_away(score, prior):

    if score < prior: score = prior
    score_no_prior = (score - prior) / (1 - prior)

    return score_no_prior

In [3]:
prior = 0.041

In [12]:
# Subscores 

neighborhood = 0
neighborhood_transferred = 0
fusion = 0
cooccurrence = 0
homology = 0
coexpression = 0
coexpression_transferred = 0
experiments = 0
experiments_transferred = 0
database = 0
database_transferred = 0
textmining = 0
textmining_transferred = 0
initial_combined = 0

In [13]:
## divide by 1000

neighborhood = float(neighborhood) / 1000
neighborhood_transferred = float(neighborhood_transferred) / 1000
fusion = float(fusion) / 1000
cooccurrence =  float(cooccurrence) / 1000
homology = float(homology) / 1000
coexpression = float(coexpression) / 1000
coexpression_transferred = float(coexpression_transferred) / 1000
experiments = float(experiments) / 1000
experiments_transferred = float(experiments_transferred) / 1000
database = float(database) / 1000
database_transferred = float(database_transferred) / 1000
textmining = float(textmining) / 1000
textmining_transferred = float(textmining_transferred) / 1000
initial_combined = int(initial_combined)

In [14]:
## compute prior away

neighborhood_prior_corrected                 = compute_prior_away (neighborhood, prior)             
neighborhood_transferred_prior_corrected     = compute_prior_away (neighborhood_transferred, prior) 
fusion_prior_corrected                       = compute_prior_away (fusion, prior)             
cooccurrence_prior_corrected                 = compute_prior_away (cooccurrence, prior)           
coexpression_prior_corrected                 = compute_prior_away (coexpression, prior)            
coexpression_transferred_prior_corrected     = compute_prior_away (coexpression_transferred, prior) 
experiments_prior_corrected                  = compute_prior_away (experiments, prior)   
experiments_transferred_prior_corrected      = compute_prior_away (experiments_transferred, prior) 
database_prior_corrected                     = compute_prior_away (database, prior)      
database_transferred_prior_corrected         = compute_prior_away (database_transferred, prior)
textmining_prior_corrected                   = compute_prior_away (textmining, prior)            
textmining_transferred_prior_corrected       = compute_prior_away (textmining_transferred, prior)

print("neighborhood_prior_corrected:", neighborhood_prior_corrected)                        
print("neighborhood_transferred_prior_corrected:", neighborhood_transferred_prior_corrected)
print("fusion_prior_corrected:", fusion_prior_corrected)            
print("cooccurrence_prior_corrected:", cooccurrence_prior_corrected)                         
print("coexpression_prior_corrected:", coexpression_prior_corrected)                        
print("coexpression_transferred_prior_corrected:", coexpression_transferred_prior_corrected)    
print("experiments_prior_corrected:", experiments_prior_corrected)    
print("experiments_transferred_prior_corrected:", experiments_transferred_prior_corrected)     
print("database_prior_corrected:", database_prior_corrected)          
print("database_transferred_prior_corrected:", database_transferred_prior_corrected)        
print("textmining_prior_corrected:", textmining_prior_corrected)             
print("textmining_transferred_prior_corrected:", textmining_transferred_prior_corrected)    

neighborhood_prior_corrected: 0.0
neighborhood_transferred_prior_corrected: 0.0
fusion_prior_corrected: 0.0
cooccurrence_prior_corrected: 0.0
coexpression_prior_corrected: 0.0
coexpression_transferred_prior_corrected: 0.0
experiments_prior_corrected: 0.0
experiments_transferred_prior_corrected: 0.0
database_prior_corrected: 0.0
database_transferred_prior_corrected: 0.0
textmining_prior_corrected: 0.0
textmining_transferred_prior_corrected: 0.0


In [15]:
## then, combine the direct and transferred scores for each category:

neighborhood_both_prior_corrected = 1.0 - (1.0 - neighborhood_prior_corrected) * (1.0 - neighborhood_transferred_prior_corrected)
coexpression_both_prior_corrected = 1.0 - (1.0 - coexpression_prior_corrected) * (1.0 - coexpression_transferred_prior_corrected)
experiments_both_prior_corrected = 1.0 - (1.0 - experiments_prior_corrected) * (1.0 - experiments_transferred_prior_corrected)
database_both_prior_corrected = 1.0 - (1.0 - database_prior_corrected) * (1.0 - database_transferred_prior_corrected)
textmining_both_prior_corrected = 1.0 - (1.0 - textmining_prior_corrected) * (1.0 - textmining_transferred_prior_corrected)

print("neighborhood_both_prior_corrected:", neighborhood_both_prior_corrected)    
print("coexpression_both_prior_corrected:", coexpression_both_prior_corrected)    
print("experiments_both_prior_corrected:", experiments_both_prior_corrected)    
print("database_both_prior_corrected:", database_both_prior_corrected)    
print("textmining_both_prior_corrected:", textmining_both_prior_corrected)    

neighborhood_both_prior_corrected: 0.0
coexpression_both_prior_corrected: 0.0
experiments_both_prior_corrected: 0.0
database_both_prior_corrected: 0.0
textmining_both_prior_corrected: 0.0


In [16]:
## now, do the homology correction on cooccurrence and textmining:

cooccurrence_prior_homology_corrected = cooccurrence_prior_corrected * (1.0 - homology)
textmining_both_prior_homology_corrected = textmining_both_prior_corrected * (1.0 - homology)

print("cooccurrence_prior_homology_corrected:", cooccurrence_prior_homology_corrected)    
print("textmining_both_prior_homology_corrected:", textmining_both_prior_homology_corrected)    

cooccurrence_prior_homology_corrected: 0.0
textmining_both_prior_homology_corrected: 0.0


In [17]:
## next, do the 1 - multiplication:

combined_score_one_minus = (
    (1.0 - neighborhood_both_prior_corrected) *
    (1.0 - fusion_prior_corrected) *
    (1.0 - cooccurrence_prior_homology_corrected) *
    (1.0 - coexpression_both_prior_corrected) *
    (1.0 - experiments_both_prior_corrected) *
    (1.0 - database_both_prior_corrected) *
    (1.0 - textmining_both_prior_homology_corrected) ) 

print("combined_score_one_minus:", combined_score_one_minus)

combined_score_one_minus: 1.0


In [18]:
## and lastly, do the 1 - conversion again, and put back the prior *exactly once*

combined_score = (1.0 - combined_score_one_minus)            ## 1- conversion
combined_score *= (1.0 - prior)                              ## scale down
combined_score += prior                                      ## and add prior.
print("combined_score:", combined_score)

combined_score: 0.041


In [19]:
## round

combined_score = int(combined_score * 1000)
print("combined_score:", combined_score)

combined_score: 41
